In [6]:
!pip freeze | grep scikit-learn

scikit-learn==1.5.0


In [7]:
!python -V

Python 3.11.11


In [22]:
import pickle
import pandas as pd
import os

In [18]:
year = 2023
month = 3

In [9]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [10]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [19]:
df = read_data(f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet')

In [14]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

### Standard Deviation

In [15]:
print("standard deviation of the predicted duration:", y_pred.std())

standard deviation of the predicted duration: 6.247488852238704


### Preparing Output

In [20]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
df_results = df[["ride_id"]]
df_results["y_pred"] = y_pred

/var/folders/pm/qyg6bv9j25q7zp4_b3mt8j_c0000gn/T/ipykernel_4009/3667852093.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_results["y_pred"] = y_pred


In [21]:
output_file = "output.parquet"

df_results.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [23]:
# Check size
file_size = os.path.getsize(output_file) / (1024 * 1024)
print("Size of file in MB", file_size)

Size of file in MB 65.4610595703125


### Command to create script

```
jupyter nbconvert starter.ipynb --to script
```